### Develop Python code that replicates the bioluminsence proxy calculations 

This Notebook is part of the auv-python project (private repository at https://github.com/mbari-org/auv-python). It demonstrates how to calculate the bioluminescence proxy variables from the raw data. These variables are 'adinos', 'bg_biolum', 'diatoms', 'fluo', 'hdinos', 'intflash', 'nbflash_high', 'nbflash_low', and 'profile'. In the production auv-python code implemented from the deomonstration the text "biolume_" preceeds these names in the netCDF file produced by the add_biolume_proxies() method in [resample.py](https://github.com/mbari-org/auv-python/blob/main/src/data/resample.py).

To execute it (for example):

```bash
    # Mount cifs://atlas.shore.mbari.org/AUVCTD to /Volumes/AUVCTD (on Mac)
    cd dev
    git clone https://github.com/mbari-org/auv-python.git
    cd auv-python
    poetry install
    poetry shell
    src/data/process_dorado.py --mission 2021.102.02 -v --no_cleanup
    jupyter notebook
    # Then open this notebook and run it
```

The calculations derive from Appendix B of Messié et al. 2019 (https://doi.org/10.1016/j.pocean.2018.12.010). The Matlab code is at https://bitbucket.org/messiem/toolbox_blprocess/src/master/)


In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join("../src/data"))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import xarray as xr
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH, MISSIONNETCDFS

# Assumes that data have been processed locally with:
# src/data/process_dorado.py --mission 2021.102.02 -v --no_cleanup
# (a Diamond mission with lots of peak flashes at head of Soquel Canyon)
# Share this view URL for this mission: http://stoqs.mbari.org/p/w2UREyQ
auv_name = "dorado"
mission = "2021.102.02"

In [ ]:
# Load full resolution data from the _align.nc file into an xarray Dataset
netcdfs_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS, mission)
opendap_base = os.path.join("http://dods.mbari.org:8181/opendap/data/auvctd/surveys/", mission.split('.')[0], 'netcdf')
# Use `opendap_base` with port 8181 to test new hyrax-docker opendap server
ds_align = xr.open_dataset(os.path.join(netcdfs_dir, f"{auv_name}_{mission}_align.nc"))
ds_1S = xr.open_dataset(os.path.join(opendap_base, f"{auv_name}_{mission}_1S.nc"))
print("New hyrax-docker link: " + os.path.join(opendap_base, f"{auv_name}_{mission}_align.nc"))
ds_align

In [ ]:
# Focus on a 10 minute period that is here in STOQS: http://stoqs.mbari.org/p/V3F_1d0
# Do not commit to the repo the output of this cell and the ones below, they are too large
hv.extension("bokeh")   # See https://github.com/holoviz/holoviews/issues/4861#issuecomment-1239739743
subset_ds = ds_align[["biolume_raw"]].sel(biolume_time60hz=slice("2021-04-13 10:00:00", "2021-04-13 10:10:00"))
df = subset_ds.to_pandas()
raw_10m_plot = subset_ds["biolume_raw"].to_pandas().rename("raw").hvplot(color="grey", width=1000, height=300, title="Raw 10 minute sample data")
# print(df.describe())
raw_10m_plot

In [ ]:
import pandas as pd
import rolling

def compute_bg_envelope(use_pandas_rolling, envelope_mini=1.5e10, flash_threshold=1.5e11):
    # (1) Dinoflagellate and zooplankton proxies
    # Comparable to https://bitbucket.org/messiem/toolbox_blprocess/src/master/bl_proxies_biolum.m

    from BLFilter import Filter # Borrowed from https://bitbucket.org/mbari/pybl/src/master/biolum/BLFilter.py
                                # See also https://bitbucket.org/mbari/pybl/src/master/test/biolum/BLFilter_test.py
    from scipy import signal

    sample_rate = 60
    env_size = 5 * sample_rate          # number is width of the filter in seconds
    smooth_size = 5 * sample_rate      # number is width of the filter in seconds

    # Compute background biolumenesence envelope
    filt = Filter(window_size=env_size, target_record_size=len(df))

    if use_pandas_rolling:
        min_bg_unsmoothed = df["biolume_raw"].rolling(env_size, min_periods=0, center=True).min()
        min_bg = min_bg_unsmoothed.rolling(smooth_size, min_periods=0, center=True).mean().values
        title = "Background envelope and peaks for 10 minute sample data - Pandas rolling"
    else:
        # BLFilter.Filter.apply_filter() takes 30 seconds for a mission - Pandas is likely faster
        min_bg_unsmoothed, min_bg = filt.apply_filter((df["biolume_raw"], []), rolling.Min)
        title = "Background envelope and peaks for 10 minute sample data - BLFilter"

    min_bg_unsmoothed_plot = pd.Series(min_bg_unsmoothed, index=df.index).rename("min_bg_unsmoothed").hvplot(
        width=1000, height=300, title=title
    )
    min_bg_plot = pd.Series(min_bg, index=df.index).rename("min_bg").hvplot()

    if use_pandas_rolling:
        med_bg_unsmoothed = df["biolume_raw"].rolling(env_size, min_periods=0, center=True).median()
        med_bg = med_bg_unsmoothed.rolling(smooth_size, min_periods=0, center=True).mean().values
    else:
        # BLFilter.Filter.apply_filter() takes 30 seconds for a mission - Pandas is likely faster
        med_bg_unsmoothed, med_bg = filt.apply_filter((df["biolume_raw"], []), rolling.Median)

    med_bg_unsmoothed_plot = pd.Series(med_bg_unsmoothed, index=df.index).rename("med_bg_unsmoothed").hvplot()
    med_bg_plot = pd.Series(med_bg, index=df.index).rename("med_bg").hvplot()

    max_bg = med_bg * 2.0 - min_bg
    max_bg_orig_plot = pd.Series(max_bg, index=df.index).rename("max_bg_orig").hvplot()

    # The Matlab to apply envelope_mini: 
    # minimum value for the envelope (max_bgrd - med_bgrd) to avoid very dim flashes when the background is low (default 1.5E10 ph/s)
    # BP.max_bgrd=2*BP.med_bgrd-BP.min_bgrd;
    # ilow=BP.max_bgrd-BP.med_bgrd<envelope_mini; 
    # BP.max_bgrd(ilow)=BP.med_bgrd(ilow)+envelope_mini;
    max_bg[max_bg - med_bg < envelope_mini] = med_bg[max_bg - med_bg < envelope_mini] + envelope_mini
    max_bg_plot = pd.Series(max_bg, index=df.index).rename("max_bg").hvplot(color="orange")

    # Find the high and low peaks
    peaks, _ = signal.find_peaks(df["biolume_raw"], height=max_bg)
    s_peaks = pd.Series(df["biolume_raw"][peaks], index=df.index[peaks])
    nbflash_high = s_peaks[s_peaks > flash_threshold]
    nbflash_low = s_peaks[s_peaks <= flash_threshold]
    nbflash_high_plot = nbflash_high.rename("nbflash_high").hvplot(kind="scatter", color="red", marker="star", s=50)
    nbflash_low_plot = nbflash_low.rename("nbflash_low").hvplot(kind="scatter", color="goldenrod", marker="star", s=50)

    # Plot everything together
    plots = min_bg_unsmoothed_plot * min_bg_plot * med_bg_plot * max_bg_orig_plot * max_bg_plot * raw_10m_plot * nbflash_high_plot * nbflash_low_plot
    return med_bg, max_bg, min_bg, nbflash_high, nbflash_low, plots

In [ ]:
%%time
# Visually compare the results of BLFilter.Filter.apply_filter() and Pandas rolling functions
med_bg, max_bg, min_bg, nbflash_high, nbflash_low, plots = compute_bg_envelope(use_pandas_rolling=False)
# print(pd.DataFrame(med_bg).describe())
# print(med_bg.min())
plots

In [ ]:
%%time
# Visually compare the results of BLFilter.Filter.apply_filter() and Pandas rolling functions
med_bg, max_bg, min_bg, nbflash_high, nbflash_low, plots = compute_bg_envelope(use_pandas_rolling=True)
plots

In [ ]:
# Construct full time series of background BL and flashes with NaNs for non-flash values
s_nbflash_high = pd.Series(np.nan, index=df.index)
s_nbflash_high.loc[nbflash_high.index] = nbflash_high

s_nbflash_low = pd.Series(np.nan, index=df.index)
s_nbflash_low.loc[nbflash_low.index] = nbflash_low

# Count the number of flashes per second - use 15 second stepping window
sample_rate = 60
flash_count_seconds = 15
flash_window = flash_count_seconds * sample_rate
nbflash_high_counts = s_nbflash_high.rolling(flash_window, step=sample_rate, min_periods=0, center=True).count().resample("1S").mean() / flash_count_seconds
nbflash_low_counts = s_nbflash_low.rolling(flash_window, step=sample_rate, min_periods=0, center=True).count().resample("1S").mean() / flash_count_seconds

nbflash_high_counts_plot = nbflash_high_counts.rename("nbflash_high").hvplot(width=1000, height=300, title="Counts of flashes per second", color="red")
nbflash_low_counts_plot = nbflash_low_counts.rename("nbflash_low").hvplot(color="goldenrod")
# nbflash_high_counts_plot * nbflash_low_counts_plot


In [ ]:
# Compute flashes per liter - Units: flashes per liter = (flashes per second / mL/s) * 1000 mL/L
flow = ds_align[["biolume_flow"]].sel(biolume_time=slice("2021-04-13 10:00:00", "2021-04-13 10:10:00"))["biolume_flow"].to_pandas().resample("1S").mean()
nbflash_high_per_liter = nbflash_high_counts.divide(flow) * 1000
nbflash_low_per_liter = nbflash_low_counts.divide(flow) * 1000

nbflash_high_per_liter_plot = nbflash_high_per_liter.rename("nbflash_high").hvplot(width=1000, height=300, title="Counts of flashes per liter", color="red")
nbflash_low_per_liter_plot = nbflash_low_per_liter.rename("nbflash_low").hvplot(color="goldenrod")
# nbflash_high_per_liter_plot * nbflash_low_per_liter_plot

In [ ]:
# Compare with netcdf_proxies file
netcdf_proxy = f"http://odss.mbari.org/thredds/dodsC/Other/routine/Products/Dorado/netcdf_proxies/{mission.split('.')[0]}/Dorado_{mission.replace('.', '_')}_proxies.nc"
ds_pr = xr.open_dataset(netcdf_proxy)
ds_pr_subset = ds_pr.sel(time=slice("2021-04-13 10:00:00", "2021-04-13 10:10:00"))
ds_pr_nbflash_high = ds_pr_subset["nbflash_high"].to_pandas().rename('pr_nbflash_high')
ds_pr_nbflash_high_plot = ds_pr_nbflash_high.hvplot(width=1000, height=300, title=f"Compare with netcdf_proxies file (dashed lines)", color="darkred").options(line_dash='dashed')
ds_pr_nbflash_low = ds_pr_subset["nbflash_low"].to_pandas().rename('pr_nbflash_low')
ds_pr_nbflash_low_plot = ds_pr_nbflash_low.hvplot(color="darkgoldenrod").options(line_dash='dashed')

ds_pr_nbflash_high_plot * nbflash_high_per_liter_plot * ds_pr_nbflash_low_plot * nbflash_low_per_liter_plot

In [ ]:
# Create Fig. 5a in the paper - Histogram of bg_BL/fluo ratio for the whole mission
# Compute the ratio of nighttime background BL to fluorescence from the 1S.nc file
fluo = ds_1S["hs2_fl700"].to_pandas()
fluo[fluo < 0] = 0.0    # Replace negative values with 0
bg_BL = ds_1S["biolume_bg_biolume"].to_pandas()     # Compare with _1S.nc file

# Remove zeros and infinities
bg_bl_fluo_ratio = bg_BL.divide(fluo).replace([0.0, np.inf, -np.inf], np.nan).dropna()
proxy_ratio_adinos = bg_BL.quantile(0.99) / fluo.quantile(0.99)
print(f"proxy_ratio_adinos = {proxy_ratio_adinos:.2e}, log10(proxy_ratio_adinos) = {np.log10(proxy_ratio_adinos):.2f}, computed from 99th percentile of bg_BL and fluo (orange on plot)")

# Histogram of log10 of bg_BL/fluo ratio
log10_bg_bl_fluo_ratio = np.log10(bg_bl_fluo_ratio)
print(f"proxy_ratio_adinos = {bg_bl_fluo_ratio.quantile(.5):.2e}, log10(proxy_ratio_adinos) = {log10_bg_bl_fluo_ratio.quantile(.5):.2f}, computed from median of log10_bg_bl_fluo_ratio (red on plot)")
# print(log10_bg_bl_fluo_ratio.describe())
log10_bg_bl_fluo_ratio_hist = log10_bg_bl_fluo_ratio.rename("log10_bg_bl_fluo_ratio").hvplot.hist(width=1000, height=300, title="Fig. 5a", bins=100)
proxy_ratio_adinos_curve = hv.Curve([(np.log10(proxy_ratio_adinos), 0), (np.log10(proxy_ratio_adinos), 1700)]).opts(color="orange", line_width=2)
proxy_ratio_adinos_curve2 = hv.Curve([(log10_bg_bl_fluo_ratio.quantile(.5), 0), (log10_bg_bl_fluo_ratio.quantile(.5), 1700)]).opts(color="red", line_width=2)
log10_bg_bl_fluo_ratio_hist * proxy_ratio_adinos_curve * proxy_ratio_adinos_curve2

In [ ]:
# Fig. 5b from the paper - scatter plot of bg_BL vs. fluo
scatter_plot = hv.Points((fluo, bg_BL)).opts(width=500, height=400, title="Fig. 5b", xlabel="fluo", ylabel="bg_BL")
slope_plot = hv.Curve([(0, 0), (fluo.quantile(0.99), bg_BL.quantile(0.99))]).opts(color="red")
scatter_plot * slope_plot

In [ ]:
# Compare envelope values with netcdf_proxies file bg_biolum values
s_med_bg = pd.Series(med_bg, index=df.index).resample("1S").mean().divide(flow) * 1000
s_max_bg = pd.Series(max_bg, index=df.index).resample("1S").mean().divide(flow) * 1000
s_min_bg = pd.Series(min_bg, index=df.index).resample("1S").mean().divide(flow) * 1000

s_med_bg_plot = s_med_bg.rename("med_bg").hvplot(width=1000, height=300, title="Compare with netcdf_proxies file (dashed lines)")
s_max_bg_plot = s_max_bg.rename("max_bg").hvplot(color="red")
s_min_bg_plot = s_min_bg.rename("min_bg").hvplot(color="green")

pr_bg_biolum_plot = ds_pr_subset["bg_biolum"].to_pandas().rename('pr_bg_biolume').hvplot().options(line_dash='dashed')

s_med_bg_plot * s_max_bg_plot * s_min_bg_plot * pr_bg_biolum_plot

In [ ]:
# Compare fluo with netcdf_proxies file values
s_fluo = pd.Series(fluo, index=s_med_bg.index).rename("fluo")
pr_fluo_plot = ds_pr_subset["fluo"].to_pandas().rename('pr_fluo').hvplot(width=1000, height=300, title=f"Compare with netcdf_proxies file (dashed lines)", color="darkred").options(line_dash='dashed')
fluo_plot = s_fluo.hvplot()
pr_fluo_plot * fluo_plot

In [ ]:
# Compute intflash (small jellies) and copare with netcdf_proxies file values
# Inperpolate the med_bg values to match the biolume_raw values and compute intflash
s_med_bg_60 = pd.Series(np.interp(df["biolume_raw"].index, s_med_bg.index, s_med_bg.values), index=df["biolume_raw"].index)
s_intflash = (df["biolume_raw"] - s_med_bg_60).rolling(flash_window, min_periods=0, center=True).max().resample("1S").mean()
intflash_plot = s_intflash.rename('intflash').hvplot(color="darkblue", width=1000, height=300, title=f"Compare intflash (small jellies) and netcdf_proxies file (dashed lines)")
pr_intflash_plot = ds_pr_subset["intflash"].to_pandas().rename('pr_intflash').hvplot(color="darkgoldenrod").options(line_dash='dashed')

intflash_plot * pr_intflash_plot


In [ ]:
# (2) H-dino, a-dino and a-other proxies
# See https://bitbucket.org/mbari/pybl/src/master/biolum/BLProxies.py

# From: https://bitbucket.org/messiem/toolbox_blprocess/src/master/bl_proxies_fluobiolum.m
#	fluo: fluorescence (proxy for phytoplankton = adinos + aother)
#	bgrd_BL: background bioluminescence (proxy for dinoflagellates)
# 	ratioAdinos: typical bgrd_BL/fluo ratio for dinoflagellates populations, typically identified from an histogram over an entire dataset
#	calfactor: possible calibration to normalize the proxies (typically fluorescence 99th percentile). 
#		       If not given, no calibration is applied (calfactor=1) and the proxies are given in fluorescence units.
# Values from https://bitbucket.org/mbari/pybl/src/master/config/pybl_config.yml
# proxy_ratio_adinos = 2.65E+10
# proxy_cal_factor = 11.6739
# Using entire Diamond data set in 5.0-mpm-stoqs2parquet.ipynb: df['hs2_fl700'].quantile(0.99) = 0.0064926
# From comparing with values in netcdf_proxy file: proxy_cal_factor = 0.00470
#proxy_cal_factor = 0.0064926
proxy_cal_factor = 0.00470
proxy_ratio_adinos = bg_BL.quantile(0.99) / fluo.quantile(0.99)
print(f"Using proxy_ratio_adinos = {proxy_ratio_adinos:.2e}, computed from 99th percentile of bg_BL and fluo")
print(f"Using proxy_cal_factor = {proxy_cal_factor}")

# Compute the proxies using the hs2_fl700 for fluorescence
pseudo_fluorescence = s_med_bg / proxy_ratio_adinos
fluo = ds_align["hs2_fl700"].sel(hs2_time=slice("2021-04-13 10:00:00", "2021-04-13 10:10:00")).to_pandas().resample('1S').mean()
adinos = np.minimum(fluo.values, pseudo_fluorescence) / proxy_cal_factor
hdinos = (pseudo_fluorescence - np.minimum(fluo.values, pseudo_fluorescence)) / proxy_cal_factor
diatoms = (fluo.values - pseudo_fluorescence) / proxy_cal_factor    # For Monterey Bay aother is diatoms

# Plot the proxies
adinos_plot = pd.Series(adinos, index=s_med_bg.index).rename("adinos").hvplot(width=1000, height=300, color="green", title="Compare fluobiolum proxies")
hdinos_plot = pd.Series(hdinos, index=s_med_bg.index).rename("hdinos").hvplot(color="blue")
diatoms_plot = pd.Series(diatoms, index=s_med_bg.index).rename("diatoms").hvplot(color="red")

# Compare with netcdf_proxy file
pr_adinos_plot = ds_pr_subset["adinos"].to_pandas().rename("pr_adinos").hvplot().options(line_dash="dashed", color="green")
pr_hdinos_plot = ds_pr_subset["hdinos"].to_pandas().rename("pr_hdinos").hvplot().options(line_dash="dashed", color="blue")
pr_diatoms_plot = ds_pr_subset["diatoms"].to_pandas().rename("pr_diatoms").hvplot().options(line_dash="dashed", color="red")

# Compare with proxies loaded for this 10 minute period into stoqs_all_dorado: http://stoqs.mbari.org/p/uECQh14
adinos_plot * hdinos_plot * diatoms_plot * pr_adinos_plot * pr_hdinos_plot * pr_diatoms_plot

In [ ]:
# Use Douglas-Peucker algorithm to simplify the depth-time curve, identifying inflection points
from utils import simplify_points

# Make list of epoch second and depth tuples to pass to simplify_points
points = []
for time, depth in zip(ds_1S["time"].astype(np.float64).values/1.e9, ds_1S["depth"].values):
    points.append((time, depth))

simplify_criteria = 10  # 100 is course, .001 is fine - 10 is about right for typical diamond missions
simple_depth = simplify_points(points, simplify_criteria)

s_simple_depth = pd.Series([d for t,d,k in simple_depth], index=[pd.to_datetime(t, unit='s') for t,d,k in simple_depth]).rename("simple_depth")
s_simple_depth_line = s_simple_depth.hvplot(width=1000, height=300, title="Identifying inflection points in depth values", )
s_simple_depth_scatter = s_simple_depth.hvplot(kind="scatter", width=1000)

# Original depth plot
orig_depth_plot = ds_1S["depth"].to_pandas().rename("orig_depth").hvplot(width=1000, height=300)

s_simple_depth_line * s_simple_depth_scatter * orig_depth_plot

In [ ]:
# Assign profile numbers to each time value using specified depth threshold in meters
depth_threshold = 15
profiles = []
count = 1
k = 0
for tv in ds_1S["time"].values:
    if tv > s_simple_depth.index[k+1]:
        # Encountered a new simple_depth point
        k += 1
        if abs(s_simple_depth[k+1] - s_simple_depth[k]) > depth_threshold:
            # Completed downcast or upcast
            count += 1
    profiles.append(count)
    if k > len(s_simple_depth) - 2:
        break

s_profile = pd.Series(profiles, index=ds_1S["time"])
s_profile_plot = s_profile.rename("s_profile").hvplot(width=1000, height=400, title="Compare profile numbers")

# Compare with netcdf_proxy file
s_pr_profile_plot = ds_pr["profile"].to_pandas().rename('pr_profile').hvplot()
diffs = (s_profile - ds_pr["profile"].to_pandas()).dropna().diff().replace([0], np.nan).dropna()
diffs_plot = diffs.rename('diffs').hvplot.scatter(color="red")

orig_depth_plot * s_profile_plot * s_simple_depth_scatter * s_pr_profile_plot * diffs_plot